# 📈 Black-Scholes Option Pricing Demo (Interactive)

This notebook demonstrates how to price European Call and Put options using the **Black-Scholes model**, implemented via **QuantLib**.
We use interactive sliders to experiment with key parameters like volatility, interest rate, and time to maturity.

---


## 🧠 What is the Black-Scholes Model?

The **Black-Scholes Model (BSM)** is a mathematical framework used to estimate the theoretical price of European-style options.
It assumes a log-normal distribution of asset prices and constant volatility and interest rate.

### ⚙️ Core Formula:

For a Call Option:
$$ C = S_0 N(d_1) - K e^{-rT} N(d_2) $$

For a Put Option:
$$ P = K e^{-rT} N(-d_2) - S_0 N(-d_1) $$

Where:
- \( S_0 \): Spot price
- \( K \): Strike price
- \( T \): Time to maturity (in years)
- \( r \): Risk-free interest rate
- \( \sigma \): Volatility of the underlying
- \( N \): Cumulative distribution function of the standard normal distribution

> We'll be using QuantLib's `BlackScholesMertonProcess` and `EuropeanOption` classes to perform this calculation.

---


In [2]:
import QuantLib as ql
import ipywidgets as widgets
from IPython.display import display

In [3]:
def price_european_option(option_type, S, K, T, r, sigma):
    maturity_date = ql.Date().todaysDate() + int(T * 365)
    calendar = ql.TARGET()
    day_count = ql.Actual365Fixed()

    spot_handle = ql.QuoteHandle(ql.SimpleQuote(S))
    flat_ts = ql.YieldTermStructureHandle(ql.FlatForward(0, calendar, r, day_count))
    flat_vol_ts = ql.BlackVolTermStructureHandle(ql.BlackConstantVol(0, calendar, sigma, day_count))

    bsm_process = ql.BlackScholesProcess(spot_handle, flat_ts, flat_vol_ts)

    option_type_ql = ql.Option.Call if option_type == "Call" else ql.Option.Put
    payoff = ql.PlainVanillaPayoff(option_type_ql, K)
    exercise = ql.EuropeanExercise(maturity_date)
    european_option = ql.VanillaOption(payoff, exercise)

    european_option.setPricingEngine(ql.AnalyticEuropeanEngine(bsm_process))
    return european_option.NPV()


## 🎛️ Interactive Inputs

Use the sliders below to change:
- Spot Price (S)
- Strike Price (K)
- Time to Maturity (T)
- Interest Rate (r)
- Volatility (σ)

Select **Call or Put** from the dropdown to switch between option types.

The output below shows the real-time price computed using the Black-Scholes model.

---


In [4]:
option_type = widgets.Dropdown(options=['Call', 'Put'], description='Option:')
S = widgets.FloatSlider(value=100, min=50, max=200, step=1, description='Spot (S):')
K = widgets.FloatSlider(value=100, min=50, max=200, step=1, description='Strike (K):')
T = widgets.FloatSlider(value=1, min=0.01, max=2, step=0.01, description='Time (T in years):')
r = widgets.FloatSlider(value=0.05, min=0.0, max=0.15, step=0.005, description='Rate (r):')
sigma = widgets.FloatSlider(value=0.2, min=0.05, max=1.0, step=0.01, description='Vol (σ):')

ui = widgets.VBox([option_type, S, K, T, r, sigma])
out = widgets.Output()

def update_option_price(*args):
    with out:
        out.clear_output()
        price = price_european_option(option_type.value, S.value, K.value, T.value, r.value, sigma.value)
        print(f"📈 {option_type.value} Option Price = ₹ {price:.2f}")

for widget in [option_type, S, K, T, r, sigma]:
    widget.observe(update_option_price, names='value')

display(ui, out)
update_option_price()


Output()

## 📊 Interpretation

Once you adjust the sliders:
- The **option price** displayed is the theoretical value as per the BSM model.
- You can observe:
  - How **increasing volatility** inflates option price (more uncertainty = more value).
  - How **time decay** reduces price as T → 0.
  - Effect of **interest rate** on Call/Put differently (Call ↑ with r, Put ↓).

⚠️ Note: This is a **theoretical price**, assuming continuous trading, frictionless markets, and constant volatility — which often deviates in real-world conditions.

---


## 📌 What's Next?

We plan to extend this notebook with:
- Real option chain comparison: **Market vs BSM**
- Implied Volatility (IV) estimation from traded options
- Monte Carlo simulation of payoff distribution
- Greeks estimation: Delta, Gamma, Vega

Stay tuned at 👉 [Quantfolio GitHub](https://github.com/adas-quant/quantfolio)

---
